### sql-cheatsheets
![sql cheat sheet page1](sql-page-1.png)

![sql cheat sheet page2](sql-page-2.png)

### Review Banking DB

[schema](banks/schema.sql)

[seed](banks/seed.sql)

### Query

-- who are the customer paid through Wells Fargo?




[Sakila DB](https://dev.mysql.com/doc/sakila/en/sakila-structure-tables.html)